<a href="https://colab.research.google.com/github/Marine27/TER/blob/master/PatternMining/Process_for_structure_study_pm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Études *Pattern Mining*

### Imports

In [0]:
import pandas as pd  
import numpy as np
import os

In [0]:
def group_age(ages):
  """
    Groupe les âges par année et mois
  """
  ages_group = []
  for i in range(len(ages)) :
    mois = int(ages[i][2:4])
    jours = int(ages[i][-2:])
    if jours < 15 :
      ages_group.append(ages[i][:4])
    else:
      mois += 1
      if mois < 10 :
        ages_group.append(ages[i][:2]+'0'+str(mois))
      else:
        ages_group.append(ages[i][:2]+str(mois))
  return ages_group
#######################
def group_an_age(ages):
  """
    Groupe les âges par demi-année
  """
  ages_group = []
  for i in range(len(ages)) :
    an = int(ages[i][0])
    mois = int(ages[i][2:4])
    if mois < 6 :
      ages_group.append(str(an)+'_00')
    else:
      ages_group.append(str(an)+'_06')

  return ages_group
#######################
def print_age(age):
  """
    Permet de rendre l'âge plus facilement lisible dans les visus
  """
  age_print = ''
  an = int(age[0])
  mois = int(age[2:4])
  if len(age) == 7:
    jours = int(age[-2:])
  elif len(age) == 4:
    jours = 0
  
  if an == 1:
    age_print +=str(an)+'an '
  elif an != 0:
    age_print +=str(an)+'ans '
  if mois !=0 :
    age_print +=str(mois)+'mois '
  if jours == 1:
    age_print +=str(jours)+'jour'
  elif jours != 0 :
    age_print +=str(jours)+'jours'
  return age_print

### Recuperation et preparation des donnnees

In [0]:
def load_data(nom_enfant):
  """
    Récupération et preparation des données pour un enfant donnée en paramètre
  """
  #Chargement du jeu de donnees
  data_child = pd.read_csv('/content/data_'+nom_enfant+'_final.csv',
                            sep = '\t',
                            encoding = 'utf-8',
                            index_col = False)

  # Supression de la colonne qui duplique l'index
  data_child = data_child.drop(columns = 'Unnamed: 0') 

  # Recuperation de la partie phonetique des intervention de l'enfant
  data_child = data_child.where(data_child['type']=='pho').dropna(subset=['type'])

  # Conservation des deux colonnes avec des informations pertinantes 
  data_child = data_child[['age','contenu']]

  # Creer un indexage a partir de cet ordre
  data_child['index']=[ i for i in range(len(data_child))]
  data_child= data_child.set_index('index')

  # Creation d'un numpy utile par la suite pour les boucles
  ages = data_child.age.unique()

  # Indexage par l'age de l'enfant
  age = data_child['age']

  l=[i for i in range(len(age))]
  k=[age.values]
  k.append(l)
  tuples = list(zip(*k))

  index = pd.MultiIndex.from_tuples(tuples, names=['age', 'index'])
  data_child.set_index(index,inplace=True)

  # Supression de la colonne age car present en index
  data_child=data_child[['contenu']]

  data_child['contenu']=data_child['contenu'].apply(lambda x: str(x))
  ages = list(ages)
  
  return data_child,ages

### Demarche pour decomposer les mots en phonemes et les indexer

In [0]:
IPA = "a,r,l,e,s,i,ɛ,ə,t,k,p,d,m,ɑ̃,n,u,v,o,y,ɔ̃,ʒ,ɔ,ɛ̃,f,b,j,w,ɥ,z,ø,ʃ,œ̃,œ,g,ɑ,ɲ,tr,kr,dr,gr,br,pr,fr".split(',')
ipa_1char = []
ipa_2char = []
for i in IPA :
  if len(i) !=1:
    ipa_2char.append(i)
  else:
    ipa_1char.append(i)

def cut_pho(mot):
  """
    Décortique le mot en phonème
  """
  mot = mot.lower().replace('ː', '').replace('ʔ', '').replace('›', '').replace('ʁ','r')
  i = 0
  m = []
  while i < len(mot):
    if mot[i:i+2] in list(ipa_2char):
      #print('ATTENTION',mot[i:i+2])
      m.append(mot[i:i+2])
      i += 2
    elif mot[i] in list(ipa_1char):
      m.append(mot[i])
      i += 1
    else: # permet de virer les caractere ne fesant pas partie du dic IPA
      i += 1
  return m

In [0]:
IPA_filtre = "r,t,k,p,d,f,b,g,tr,kr,dr,gr,br,pr,fr".split(',')
def pho_filtre(mot):
  """
    Décortique le mot en phonème et vérifie qu'il contient au moins un phoneme que l'on souhaite observer
  """
  for i in cut_pho(mot):
    if i in IPA_filtre:
      return cut_pho(mot)

In [0]:
def idx_pos(mot):
    """
        Indexe la position du phonème dans le mot
    """
    m = []
    for i in range(len(mot)): # On parcours chaque charactère du mot
        suff = '_d'           # On considère que le 1er charactère lu est au début du mot
        if(i>0):              # Ensuite on considère que tout les autres charactères sont en milieu de mot
            suff = '_m'
            if(i==len(mot)-1):  # Sauf pour le dernier charactère lu
                suff = '_f'   
        m.append(mot[i]+suff)   # On ajouter le phonème indexé 
    
    return m

### Addaptation de la structure

In [0]:
def structure_pm(data_child,ages):
  """
    Décortique les données en mots puis en phonèmes, 
    tout en respectant la structure nécessaire pour la librairie pymining
  """
  # Sépare en mots au lieu de phrases
  mots = {}
  for i in list(ages):
    # recupere toutes les phrases a la i eme date
    phrases = data_child['contenu'].loc[i].values
    # creer une variable texte avec toutes les phrases
    phrase = ' '.join(phrases)
    # sépare en mot
    mots[i] = phrase.split()
    for j in range(len(mots[i])):
      #mots[i][j] = [mots[i][j]] # pour etude du mot
      mots[i][j] = idx_pos(cut_pho(mots[i][j])) # pour etude des phonemes dans un mot indexé par position 
      #mots[i][j] = list(mots[i][j]) # pour etude des lettres dans un mot
    mots[i] = tuple(mots[i])

  return mots

  # se parcourt avec un age en parametre
  #mots["1_00_05"]

In [0]:
def structure_pm_filtre(data_child,ages):
  """
    Décortique les données en mots puis en phonemes, 
    tout en respectant la structure nécessaire pour la librairie pymining. 
    En conservant uniquement les mots avec des phonèmes recherchés.
  """
  # Sépare en mots au lieu de phrases
  mots = {}
  for i in list(ages):
    # recupere toutes les phrases a la i eme date
    phrases = data_child['contenu'].loc[i].values
    # creer une variable texte avec toutes les phrases
    phrase = ' '.join(phrases)
    # sépare en mot
    mots[i] = phrase.split()
    filtre = []
    for j in range(len(mots[i])):
      mots[i][j] = pho_filtre(mots[i][j])
      if mots[i][j] != None:
        filtre.append(idx_pos(mots[i][j]))
    mots[i] = tuple(filtre)

  return mots

In [0]:
def structure_pm_age_group(data_child,ages):
  """
    Décortique les données en mots puis en phonèmes avec des âges regroupés, 
    tout en respectant la structure nécessaire pour la librairie pymining
  """
  a_group = group_age(ages)
  age_unique = []
  rep_age = []
  for age in a_group:
    if age in age_unique:
      rep_age[age_unique.index(age)] +=1
    else:
      age_unique.append(age)
      rep_age.append(1)
  
  d = 0
  mots = {}
  ages = list(ages)
  for idx in range(len(rep_age)):
    fin = d+rep_age[idx]
    filtre_ages = ages[d:fin]
    a = age_unique[idx]
    mots[a] = []
    for i in filtre_ages : #groupe les ages
      # recupere toutes les phrases a la i eme date
      phrases = data_child['contenu'].loc[i].values
      # creer une variable texte avec toutes les phrases pour les annees regroupeées
      phrase = ' '.join(phrases)
    # sépare en mot
    mots[a] += phrase.split()
    for j in range(len(mots[a])):
      #mots[i][j] = [mots[i][j]] # pour etude du mot
      mots[a][j] = idx_pos(cut_pho(mots[a][j])) # pour etude des phonemes dans un mot indexé par position 
      #mots[i][j] = list(mots[i][j]) # pour etude des lettres dans un mot
    mots[a] = tuple(mots[a])
    d = fin

  return mots,age_unique

### PATTERN MINING initialisation

In [0]:
!pip install pymining
from pymining import itemmining
from pymining import assocrules
from pymining import seqmining

### Itemmining etude 1 : motif sequentiel par age par enfant

#### Itemmining

In [0]:
min_supp = 170 # Nombre de répétitions minimums requises du phonème

In [0]:
def item_mining(ages,mots):
  tab_items = []
  ages_save = []
  for age in ages:
    relim_input = itemmining.get_relim_input(mots[age])
    item_sets = itemmining.relim(relim_input, min_support=min_supp)
    if len(item_sets) != 0:
      ages_save.append(age)
      tab_item = pd.DataFrame([item_sets]).T.reset_index()
      tab_item.columns = ['item','nbr']
      
      tab_items.append(tab_item)
  ages = ages_save
  return tab_items,ages

#### Sunburst itemmining

##### Exectution pour tout les enfants

In [0]:
enfants = ['adrien','anae','antoine','julie','madeleine','theophile']
for enfant in enfants:
  globals()['data_'+enfant] , globals()['ages_'+enfant] = load_data(enfant)
  globals()['mots_'+enfant] = structure_pm(globals()['data_'+enfant],globals()['ages_'+enfant]) #sans filtre sur les pho
  #globals()['mots_'+enfant] = structure_pm_filtre(globals()['data_'+enfant],globals()['ages_'+enfant]) # avec filtre
  globals()['tab_items_'+enfant] , globals()['ages_'+enfant] = item_mining(globals()['ages_'+enfant],globals()['mots_'+enfant])
  
  # faire attention a que len('tab_items_'+enfant) soit au min de 2
  print(enfant,len(globals()['tab_items_'+enfant]),'ok')

##### Creation du fichier

In [0]:
fichier = open("pattern.json", "w")

######################
init = '{"name": "Pattern Mining",\n"children": [\n'
fichier.write(init)

######################
for enfant in enfants:
  write_enfant ='{"name": "' +enfant.capitalize()+ '",\n"children": [\n'
  fichier.write(write_enfant)
  ages = globals()['ages_'+enfant]
  tab_items = globals()['tab_items_'+enfant]
  ######################

  idx = 0
  for age in ages:
    write_age ='{"name": "' +print_age(age)+ '",\n"children": [\n'
    fichier.write(write_age)

    tab_item = tab_items[idx]
    #print(idx,age,len(tab_item),len(tab_item) > 2)

    if len(tab_item[:-1]) != 0:
      for i in range(len(tab_item[:-1])):
        item,nbr = tab_item.loc[i]
        to_write = '{ "name": "' +str(item)[11:-2]+ '", "value": ' +str(nbr)+ ' },\n'
        fichier.write(to_write)

    item,nbr = tab_item.loc[len(tab_item)-1] # -1 car l'incrementation commence a zero...
    if age != ages[-1]: # si ce n'est pas la derniere ligne du DERNIER age de cet enfant
      to_write = '{ "name": "' +str(item)[11:-2]+ '", "value": ' +str(nbr)+ '}\n]\n},\n'
      fichier.write(to_write)
      idx += 1
    else: # si c'est la DERNIERE ligne du DERNIER age de cet enfant
      if enfant != enfants[-1]: # si c'est la DERNIERE ligne du DERNIER age mais pas du dernier enfant
        to_write = '{ "name": "' +str(item)[11:-2]+ '", "value": ' +str(nbr)+ '}\n]\n}\n]\n},\n'
        fichier.write(to_write)
      else: # si c'est la DERNIERE ligne du DERNIER age du DERNIER enfant (donc la fin du doc)
        to_write = '{ "name": "' +str(item)[11:-2]+ '", "value": ' +str(nbr)+ '}\n]\n}\n]\n}\n]\n}'
        fichier.write(to_write)

fichier.close()

### Assocrules étude 2 : quel phoneme en entraine un autre ?

#### Assocrules

In [0]:
min_supp = 100 # Nombre de répétitions minimums requises du phonème
conf = 0.8 # Probabilité minimum requise pour dire que le phonème 1 entraine le phonème 2

In [0]:
def assoc_rules(ages,mots):
  tab_items = []
  ages_save = []
  for age in ages:
    relim_input = itemmining.get_relim_input(mots[age])
    item_sets = itemmining.relim(relim_input, min_support=min_supp)
    if len(item_sets) != 0:
      rules = assocrules.mine_assoc_rules(item_sets, min_support=min_supp, min_confidence=conf)
      if len(rules) != 0:
        ages_save.append(age)
        tab_item = pd.DataFrame(rules)
        tab_item.columns = ['si_pho1','alors_pho2','nbr','pourcent']
        tab_items.append(tab_item)
  ages = ages_save
  return tab_items,ages

#### Sankey assocrules

##### Exectution pour tout les enfants

In [0]:
enfants = ['adrien','anae','antoine','julie','madeleine','theophile']
for enfant in enfants:
  globals()['data_'+enfant] , globals()['ages_'+enfant] = load_data(enfant)
  #globals()['mots_'+enfant] = structure_pm(globals()['data_'+enfant],globals()['ages_'+enfant]) #sans filtre sur les pho
  globals()['mots_'+enfant] , globals()['ages_'+enfant] = structure_pm_age_group(globals()['data_'+enfant],globals()['ages_'+enfant]) #avec regroupement des ages
  globals()['tab_items_'+enfant] , globals()['ages_'+enfant] = assoc_rules(globals()['ages_'+enfant],globals()['mots_'+enfant])
  globals()['ages_'+enfant] = group_age(globals()['ages_'+enfant])
  
  # faire attention a que len('tab_items_'+enfant) soit au min de 2
  print(enfant,len(globals()['tab_items_'+enfant]),'ok')

##### Creation du fichier

In [0]:
enfants = ['adrien','anae','antoine','julie','madeleine','theophile',['adrien','anae','antoine','julie','madeleine','theophile']]
enfants
for enf in enfants:
  if enf != enfants[-1]:
    enf = [enf]
  source = []
  target = []
  value = []
  for enfant in enf:
    ages = globals()['ages_'+enfant]
    tab_items = globals()['tab_items_'+enfant]
    for i in range(len(tab_items)):
      source.append(enfant.capitalize())
      target.append(print_age(ages[i]))
      value.append(tab_items[i].nbr.sum())
      for j in range(len(tab_items[i])):
        source.append(print_age(ages[i]))
        target.append(str(tab_items[i].si_pho1[j])[11:-2]+' ')
        value.append(tab_items[i].nbr[j])

        source.append(str(tab_items[i].si_pho1[j])[11:-2]+' ')
        target.append(str(tab_items[i].alors_pho2[j])[11:-2])
        value.append(tab_items[i].nbr[j])

  data = pd.DataFrame([source,target,value]).T
  data.columns = ['source','target','value']
  #data.head()

  if enf != enfants[-1]:
    os.chdir('/content/SANKEY')
    data.to_csv('data_'+enf[0]+'.csv',
              encoding = 'utf-8',
              sep = ',',
              header = True)
  else:
    os.chdir('/content/SANKEY')
    data.to_csv('data_sankey.csv',
              encoding = 'utf-8',
              sep = ',',
              header = True)
  print(enf)

### Itemmining etude 3 : ordre d'apparition des phonemes

#### Structuration des données

In [0]:
def structure_pm_pho(data_child,ages):
  """
    Décortique les données en mots puis en phonèmes indexé, 
    et le ramène au niveau du phoneme pour la comparaison future, 
    tout en respectant la structure nécessaire pour la librairie pymining
  """
  # Sépare en mots au lieu de phrases
  phos = {}
  for i in list(ages):
    # recupere toutes les phrases a la i eme date
    phrases = data_child['contenu'].loc[i].values
    # creer une variable texte avec toutes les phrases
    phrase = ' '.join(phrases)
    # sépare en mot
    phos[i] = phrase.split()
    pho = []
    for j in range(len(phos[i])):
      pho += idx_pos(cut_pho(phos[i][j]))
    pho = ' '.join(pho)
    phos[i]=pho.split()
    pho = []
    for i in list(phos[i]):
      pho.append([i])
    phos[i]=tuple(pho)

  return phos
###########################
def crea_corpus():
  """
    Concatène toutes les data des infos préalablement
     structurées avec la fonction structure_pm_pho
  """
  data=[]
  for enfant in enfants:
    t = 160 # Permettra de creer 160*6enfants enfants ce qui fait 960seéquences
    a= len(globals()['ages_'+enfant])
    print('print1',enfant,a)
    
    for age in globals()['ages_'+enfant]:
      if len(globals()['phos_'+enfant][age])-t < t:
        del globals()['phos_'+enfant][age]
        idx = globals()['ages_'+enfant].index(age)
        del globals()['ages_'+enfant][idx]
        a -= 1
    print('print2',enfant,a)
    enfants_fictif = [[]]*t
    for age in globals()['ages_'+enfant]:
      for i in range(t):
        nbr = len(globals()['phos_'+enfant][age])%t
        if globals()['ages_'+enfant].index(age) == 0:
          enfants_fictif[i] = globals()['phos_'+enfant][age][:t]
        else :
          enfants_fictif[i] += globals()['phos_'+enfant][age][t*i:t*(i+1)]
    data += enfants_fictif
  return data

In [0]:
enfants = ['adrien','anae','antoine','julie','madeleine','theophile']
for enfant in enfants:
  globals()['data_'+enfant] , globals()['ages_'+enfant] = load_data(enfant)
  globals()['phos_'+enfant] = structure_pm_pho(globals()['data_'+enfant],globals()['ages_'+enfant])


In [0]:
data = crea_corpus()
min_supp = 960
freq_seqs = seqmining.freq_seq_enum(data,min_supp)

print(len(freq_seqs))

itemsets = []
for seq in freq_seqs:
  a,b=seq
  itemsets.append(a)

itemsets = pd.DataFrame(itemsets)
itemsets = itemsets.reset_index()
del itemsets['index']
itemsets = itemsets.sort_values(by = [0,1,3,4])

for i in range(len(itemsets.columns)):
  for j in range(len(itemsets)):
    if type(itemsets[i][j]) == str :
      itemsets[i][j] +=' '*i

itemsets.head()

#### Visu 1 Tangled tree (echec car trop de donnnees)

In [0]:
levels=[]
for i in range(len(itemsets.columns)):
  level = []
  if i == 0:
    for j in range(len(itemsets[i])):
      if type(itemsets[i][j]) == str :
        level.append("{id:'"+ itemsets[i][j] +"'}")
  else :
    for j in range(len(itemsets[i])):
      if type(itemsets[i][j]) == str :
        level.append("{id:'"+ itemsets[i][j] +"', parents:"+ str(list(filter(None,list(itemsets.T[:i][j])))) +'}')
  levels.append(level)

In [0]:
for i in range(len(levels)):
  levels[i]=sorted(set(levels[i]))

In [0]:
levels = str(levels).replace('"','').replace("}}","}").replace("{{","{")

In [0]:
fichier = open("visu.json", "w")
fichier.write(levels)
fichier.close()

#### Visu 2 Sankey

In [0]:
source = []
target = []
for i in range(1,len(itemsets.columns)):
  for j in range(len(itemsets)):
    if type(itemsets[i][j]) == str:
      source.append(itemsets[i-1][j])
      target.append(itemsets[i][j])

value = [1]*len(source)
data = pd.DataFrame([source,target,value]).T
data.columns = ['source','target','value']

In [0]:
os.chdir('/content/')
data.to_csv('data_etude3_'+str(min_supp)+'_'+str(len(freq_seqs))+'.csv',
           encoding = 'utf-8',
           sep = ',',
           header = True)